# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-01-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-01-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-08-02 04:34:47,34.223334,-82.461707,288,7,0,281.0,"Abbeville, South Carolina, US",1174.216170,2.430556
1,22001.0,Acadia,Louisiana,US,2020-08-02 04:34:47,30.295065,-92.414197,2331,71,0,2260.0,"Acadia, Louisiana, US",3756.950600,3.045903
2,51001.0,Accomack,Virginia,US,2020-08-02 04:34:47,37.767072,-75.632346,1077,15,0,1062.0,"Accomack, Virginia, US",3332.714445,1.392758
3,16001.0,Ada,Idaho,US,2020-08-02 04:34:47,43.452658,-116.241552,8004,62,0,7942.0,"Ada, Idaho, US",1662.004996,0.774613
4,19001.0,Adair,Iowa,US,2020-08-02 04:34:47,41.330756,-94.471059,20,0,0,20.0,"Adair, Iowa, US",279.642058,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,35928,35981,36036,36157,36263,36368,36471,36542,36675,36710
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,4466,4570,4637,4763,4880,4997,5105,5197,5276,5396
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,25484,26159,26764,27357,27973,28615,29229,29831,30394,30950


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1211,1225,1248,1259,1269,1270,1271,1271,1272,1283
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,123,128,134,138,144,148,150,154,157,161
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1124,1136,1146,1155,1163,1174,1186,1200,1210,1223


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,24550,24602,24793,25180,25198,25358,25389,25471,25509,25509
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2523,2608,2637,2682,2745,2789,2830,2883,2952,2961
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,17369,17369,18076,18088,18837,19233,19592,20082,20537,20988


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,337,337,337,337,349,351,354,356,356,356
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,38,38,40,40,40,40,42,42,42,44


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
1172,1065.0,Hale,Alabama,US,2020-08-02 04:34:47,32.760393,-87.632850,448,25,0,423.0,"Hale, Alabama, US",3057.811753,5.580357
2291,1107.0,Pickens,Alabama,US,2020-08-02 04:34:47,33.280949,-88.088181,359,9,0,350.0,"Pickens, Alabama, US",1801.304566,2.506964
651,1037.0,Coosa,Alabama,US,2020-08-02 04:34:47,32.936901,-86.248477,89,2,0,87.0,"Coosa, Alabama, US",834.661915,2.247191


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,4620444,154447,1461885
Brazil,2707877,93563,2037982
India,1750723,37364,1145629


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,4620444,154447,1461885,3004112,2020-08-02 04:34:47,37.904997,-91.310033
Brazil,2707877,93563,2037982,576332,2020-08-02 04:34:47,-12.669522,-48.480493
India,1750723,37364,1145629,567730,2020-08-02 04:34:47,23.088275,81.806127
Russia,843890,14034,645316,184540,2020-08-02 04:34:47,54.546312,62.120860
South Africa,503290,8153,342461,152676,2020-08-02 04:34:47,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,506613,9345,0
Florida,480028,7022,0
Texas,444738,6865,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,506613,9345,0,497268,2020-08-02 04:34:47,37.843962,-120.728594
Florida,480028,7022,0,473006,2020-08-02 04:34:47,28.940755,-82.700744
Texas,444738,6865,0,437873,2020-08-02 04:34:47,31.647790,-98.615963
New York,415767,32694,0,383073,2020-08-02 04:34:47,42.671593,-75.579694
Georgia,190012,3825,0,186187,2020-08-02 04:34:47,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,225460,23536,0
California,Los Angeles,190857,4669,0
Florida,Miami-Dade,121207,1647,0
Arizona,Maricopa,119295,2089,0
Illinois,Cook,106131,4888,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,225460,23536,0,201924,2020-08-02 04:34:47,40.767273,-73.971526,36061.0
California,Los Angeles,190857,4669,0,186188,2020-08-02 04:34:47,34.308284,-118.228241,6037.0
Florida,Miami-Dade,121207,1647,0,119560,2020-08-02 04:34:47,25.611236,-80.551706,12086.0
Arizona,Maricopa,119295,2089,0,117206,2020-08-02 04:34:47,33.348359,-112.491815,4013.0
Illinois,Cook,106131,4888,0,101243,2020-08-02 04:34:47,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,36542,5197,29831,922,1109,91,185373,38196,16903,20955,...,303910,1243,23271,17859,509,11548,10,1726,5555,3092
2020-07-31,36675,5276,30394,925,1148,91,191302,38550,17280,21130,...,304793,1264,24009,18574,558,11837,10,1728,5963,3169
2020-08-01,36710,5396,30950,925,1164,91,196543,38841,17269,21212,...,305562,1278,24783,19443,590,12160,10,1730,6228,3659


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,1271,154,1200,52,51,3,3441,728,196,718,...,46084,35,136,158,0,81,1,487,149,53
2020-07-31,1272,157,1210,52,52,3,3543,738,201,718,...,46204,35,141,164,3,82,1,493,151,67
2020-08-01,1283,161,1223,52,54,3,3596,749,201,718,...,46278,35,147,169,3,83,1,494,165,69


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,25471,2883,20082,806,395,67,80596,28366,9759,18628,...,1438,978,13680,10421,373,5016,8,856,3289,924
2020-07-31,25509,2952,20537,807,437,67,83780,28997,9983,18758,...,1439,994,14464,10421,373,5077,8,862,3803,1004
2020-08-01,25509,2961,20988,807,460,67,86499,29557,10204,18911,...,1441,1004,15299,11129,373,5324,8,862,4130,1011


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20
130,84001097,US,USA,840,1097.0,Mobile,Alabama,US,30.784723,-88.208424,...,6797,6901,7096,7193,7361,7495,7683,7879,8084,8466
109,84001055,US,USA,840,1055.0,Etowah,Alabama,US,34.045673,-86.040519,...,1495,1546,1615,1643,1690,1723,1743,1797,1857,1884
110,84001057,US,USA,840,1057.0,Fayette,Alabama,US,33.720769,-87.738866,...,128,132,134,138,144,151,156,162,163,167
111,84001059,US,USA,840,1059.0,Franklin,Alabama,US,34.442353,-87.842895,...,1051,1064,1075,1078,1098,1105,1117,1144,1162,1165
112,84001061,US,USA,840,1061.0,Geneva,Alabama,US,31.093890,-85.835728,...,167,178,183,186,192,194,198,204,210,216


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-30,85762,2879,0,170798,41759,492934,46186,49670,14689,49,...,102871,428229,39696,1407,385,88904,55803,6375,52108,2686
2020-07-31,87723,2990,0,174010,42511,500556,46791,49810,14788,49,...,105959,438293,40196,1414,406,89888,55803,6579,52940,2726
2020-08-01,89349,3136,0,177002,43173,506613,47249,49810,14877,49,...,108184,444738,40702,1421,421,90801,57541,6738,54002,2769


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-30         963    2984     539  344    658     428    733    1297   
2020-07-31         972    3056     550  355    685     430    739    1409   
2020-08-01         988    3095     554  360    704     436    743    1438   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-30          800      217  ...     111      5       56       32   
2020-07-31          806      219  ...     115      5       58       34   
2020-08-01          807      222  ...     117      5       60       36   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-30            237   331   247          0       46      5  
2020-07-31            240   335   254          0       48      5  
2020-08-01            242   348   259          0       49      5  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-30,1565,23,0,3626,442,9026,1822,4431,581,0,...,1033,6442,300,57,8,2141,1564,112,919,26
2020-07-31,1580,23,0,3694,453,9214,1838,4432,585,0,...,1060,6576,304,57,8,2174,1564,116,934,26
2020-08-01,1603,24,0,3747,458,9345,1844,4432,585,0,...,1067,6865,310,57,8,2215,1592,116,947,26


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-07-30          20      20       5    2      3      11     35       8   
2020-07-31          20      21       5    2      3      11     35       9   
2020-08-01          20      21       5    2      3      11     35       9   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-30           38        7  ...       0      0        0        0   
2020-07-31           38        7  ...       0      0        0        0   
2020-08-01           38        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-30              0     0     0         25        0      0  
2020-07-31              0     0     0         25        0      0  
2020-08-01              0     0     0         25        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,0.001947,0.018022,0.020596,0.004357,0.028757,0.0,0.035626,0.006827,0.037121,0.005036,...,0.002795,0.004850,0.030374,0.040856,0.108932,0.023396,0.0,0.008767,0.058297,0.073984
2020-07-31,0.003640,0.015201,0.018873,0.003254,0.035167,0.0,0.031984,0.009268,0.022304,0.008351,...,0.002905,0.016895,0.031713,0.040036,0.096267,0.025026,0.0,0.001159,0.073447,0.024903
2020-08-01,0.000954,0.022745,0.018293,0.000000,0.013937,0.0,0.027396,0.007549,-0.000637,0.003881,...,0.002523,0.011076,0.032238,0.046786,0.057348,0.027287,0.0,0.001157,0.044441,0.154623


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,0.000000,0.026667,0.011804,0.0,0.062500,0.0,0.046533,0.006916,0.037037,0.002793,...,0.000825,0.0,0.038168,0.012821,NaN,0.012500,0.0,0.004124,0.020548,0.292683
2020-07-31,0.000787,0.019481,0.008333,0.0,0.019608,0.0,0.029643,0.013736,0.025510,0.000000,...,0.002604,0.0,0.036765,0.037975,inf,0.012346,0.0,0.012320,0.013423,0.264151
2020-08-01,0.008648,0.025478,0.010744,0.0,0.038462,0.0,0.014959,0.014905,0.000000,0.000000,...,0.001602,0.0,0.042553,0.030488,0.0,0.012195,0.0,0.002028,0.092715,0.029851


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,0.003230,0.018728,0.025010,0.002488,0.312292,0.0,0.035206,0.019480,0.014555,0.005397,...,0.000000,0.011375,0.057432,0.00000,0.01084,0.037865,0.0,0.008245,0.001218,0.041714
2020-07-31,0.001492,0.023933,0.022657,0.001241,0.106329,0.0,0.039506,0.022245,0.022953,0.006979,...,0.000695,0.016360,0.057310,0.00000,0.00000,0.012161,0.0,0.007009,0.156279,0.086580
2020-08-01,0.000000,0.003049,0.021960,0.000000,0.052632,0.0,0.032454,0.019312,0.022138,0.008157,...,0.001390,0.010060,0.057730,0.06794,0.00000,0.048651,0.0,0.000000,0.085985,0.006972


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-30,0.023633,0.028949,NaN,0.015005,0.019308,0.016541,0.008913,0.002624,0.005958,0.0,...,0.020323,0.022038,0.012808,0.000711,0.000000,0.010353,0.014877,0.015613,0.020745,0.022070
2020-07-31,0.022866,0.038555,NaN,0.018806,0.018008,0.015463,0.013099,0.002819,0.006740,0.0,...,0.030018,0.023501,0.012596,0.004975,0.054545,0.011068,0.000000,0.032000,0.015967,0.014892
2020-08-01,0.018536,0.048829,NaN,0.017194,0.015572,0.012101,0.009788,0.000000,0.006018,0.0,...,0.020999,0.014705,0.012588,0.004950,0.036946,0.010157,0.031145,0.024168,0.020060,0.015774


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-30      0.014752  0.041536  0.001859  0.023810  0.056180  0.007059   
2020-07-31      0.009346  0.024129  0.020408  0.031977  0.041033  0.004673   
2020-08-01      0.016461  0.012762  0.007273  0.014085  0.027737  0.013953   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-07-30      0.006868  0.035116  0.005025  0.028436  ...  0.027778    0.0   
2020-07-31      0.008186  0.086353  0.007500  0.009217  ...  0.036036    0.0   
2020-08-01      0.005413  0.020582  0.001241  0.013699  ...  0.017391    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-07-30      0.244444  0.032258   0.012821  0.040881  0.008163        NaN   
2020-07-31      0.035714  0.062500   0.012658  0.012085  0.028340        NaN   
2020-08-01      0.034483  0.058824   0.008333  0.038806  0.019685        NaN   

Province_State                   
Admin2          Washakie Weston  
2020-07-30      0.022222    0.0  
2020-07-31      0.043478    0.0  
2020-08-01      0.020833    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-30,0.017555,0.045455,NaN,0.049797,0.018433,0.013247,0.000000,0.001356,0.000000,NaN,...,0.012745,0.022215,0.027397,0.017857,0.0,0.007529,0.005788,0.009009,0.008782,0.0
2020-07-31,0.009585,0.000000,NaN,0.018753,0.024887,0.020829,0.008782,0.000226,0.006885,NaN,...,0.026137,0.020801,0.013333,0.000000,0.0,0.015413,0.000000,0.035714,0.016322,0.0
2020-08-01,0.014557,0.043478,NaN,0.014348,0.011038,0.014217,0.003264,0.000000,0.000000,NaN,...,0.006604,0.043948,0.019737,0.000000,0.0,0.018859,0.017903,0.000000,0.013919,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2020-07-30         0.0    0.00    0.25  0.0    0.0     0.0    0.0  0.333333   
2020-07-31         0.0    0.05    0.00  0.0    0.0     0.0    0.0  0.125000   
2020-08-01         0.0    0.00    0.00  0.0    0.0     0.0    0.0  0.000000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-07-30          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-07-31          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-08-01          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-07-30            NaN   NaN   NaN        0.0      NaN    NaN  
2020-07-31            NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-01            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,0.002400,0.020382,0.021393,0.005992,0.043478,0.018202,0.034194,0.007004,0.035944,0.005995,...,0.002589,0.009431,0.032139,0.038446,0.068216,0.025606,2.021099e-13,0.007104,0.058446,0.055601
2020-07-31,0.003020,0.017792,0.020133,0.004623,0.039322,0.009101,0.033089,0.008136,0.029124,0.007173,...,0.002747,0.013163,0.031926,0.039241,0.082241,0.025316,1.010550e-13,0.004132,0.065947,0.040252
2020-08-01,0.001987,0.020268,0.019213,0.002311,0.026630,0.004551,0.030243,0.007842,0.014244,0.005527,...,0.002635,0.012119,0.032082,0.043013,0.069795,0.026302,5.052748e-14,0.002644,0.055194,0.097437


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,0.001538,0.025081,0.010625,5.572884e-16,0.059718,6.211124e-33,0.040905,0.007375,0.048963,0.00256,...,0.001260,0.001902,0.041719,0.022691,NaN,0.014723,0.0,0.004050,0.020338,0.172583
2020-07-31,0.001162,0.022281,0.009479,2.786442e-16,0.039663,3.105562e-33,0.035274,0.010555,0.037237,0.00128,...,0.001932,0.000951,0.039242,0.030333,NaN,0.013534,0.0,0.008185,0.016881,0.218367
2020-08-01,0.004905,0.023879,0.010111,1.393221e-16,0.039062,1.552781e-33,0.025116,0.012730,0.018618,0.00064,...,0.001767,0.000476,0.040898,0.030410,0.0,0.012865,0.0,0.005107,0.054798,0.124109


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-07-30,0.003511,0.017669,0.023208,0.001555,0.201990,0.013205,0.035357,0.018596,0.013272,0.006067,...,0.000423,0.009295,0.056411,0.015587,0.006808,0.100305,1.184238e-15,0.011677,0.045940,0.155517
2020-07-31,0.002501,0.020801,0.022932,0.001398,0.154160,0.006602,0.037431,0.020421,0.018112,0.006523,...,0.000559,0.012828,0.056860,0.007793,0.003404,0.056233,5.921189e-16,0.009343,0.101109,0.121048
2020-08-01,0.001251,0.011925,0.022446,0.000699,0.103396,0.003301,0.034943,0.019866,0.020125,0.007340,...,0.000974,0.011444,0.057295,0.037867,0.001702,0.052442,2.960595e-16,0.004672,0.093547,0.064010


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-30,0.020789,0.031820,NaN,0.014389,0.019446,0.021039,0.010460,0.004273,0.006710,3.907893e-42,...,0.021457,0.022069,0.011631,0.001192,0.013298,0.011292,0.014962,0.019382,0.018672,0.020378
2020-07-31,0.021827,0.035187,NaN,0.016597,0.018727,0.018251,0.011779,0.003546,0.006725,1.953947e-42,...,0.025737,0.022785,0.012113,0.003084,0.033922,0.011180,0.007481,0.025691,0.017319,0.017635
2020-08-01,0.020181,0.042008,NaN,0.016896,0.017150,0.015176,0.010784,0.001773,0.006372,9.769734e-43,...,0.023368,0.018745,0.012351,0.004017,0.035434,0.010669,0.019313,0.024929,0.018690,0.016705


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-07-30      0.013019  0.033962  0.005414  0.021315  0.040136  0.009615   
2020-07-31      0.011182  0.029046  0.012911  0.026646  0.040585  0.007144   
2020-08-01      0.013822  0.020904  0.010092  0.020365  0.034161  0.010549   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-07-30      0.007015  0.040527  0.007049  0.025499  ...  0.025711   
2020-07-31      0.007600  0.063440  0.007274  0.017358  ...  0.030873   
2020-08-01      0.006506  0.042011  0.004258  0.015528  ...  0.024132   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-07-30      0.000488  0.134659  0.028862   0.019561  0.039003  0.011309   
2020-07-31      0.000244  0.085187  0.045681   0.016109  0.025544  0.019824   
2020-08-01      0.000122  0.059835  0.052252   0.012221  0.032175  0.019755   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-07-30           -1.0  0.011161  0.015673  
2020-07-31           -1.0  0.027320  0.007837  
2020-08-01           -1.0  0.024077  0.003918  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-07-30,0.018414,0.032743,NaN,0.033504,0.020592,0.016830,0.002905,0.001013,0.001424,NaN,...,0.015901,0.040070,0.023202,0.008929,0.03604,0.008420,0.007460,0.016990,0.008017,0.005034
2020-07-31,0.013999,0.016371,NaN,0.026129,0.022739,0.018829,0.005843,0.000620,0.004154,NaN,...,0.021019,0.030435,0.018267,0.004464,0.01802,0.011917,0.003730,0.026352,0.012170,0.002517
2020-08-01,0.014278,0.029925,NaN,0.020238,0.016888,0.016523,0.004554,0.000310,0.002077,NaN,...,0.013812,0.037192,0.019002,0.002232,0.00901,0.015388,0.010816,0.013176,0.013044,0.001259


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                 \
Admin2           Autauga   Baldwin   Barbour      Bibb Blount   Bullock   
2020-07-30      0.000007  0.044949  0.125167  0.000008  0.500  0.000195   
2020-07-31      0.000004  0.047475  0.062583  0.000004  0.250  0.000098   
2020-08-01      0.000002  0.023737  0.031292  0.000002  0.125  0.000049   

Province_State                                              ... Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...    Park   
2020-07-30      0.001990  0.166668  0.002442  6.522579e-13  ...     NaN   
2020-07-31      0.000995  0.145834  0.001221  3.261290e-13  ...     NaN   
2020-08-01      0.000497  0.072917  0.000611  1.630645e-13  ...     NaN   

Province_State                                                             \
Admin2         Platte Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2020-07-30        NaN      NaN      NaN        NaN   NaN   NaN   0.005244   
2020-07-31        NaN      NaN      NaN        NaN   NaN   NaN   0.002622   
2020-08-01        NaN      NaN      NaN        NaN   NaN   NaN   0.001311   

Province_State                  
Admin2         Washakie Weston  
2020-07-30          NaN    NaN  
2020-07-31          NaN    NaN  
2020-08-01          NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200801,AK,3827,234806,NaN,37.0,NaN,NaN,NaN,3.0,...,238633,1,0,10f9ac619619649c68459b9c290978575b6d3db4,0,0,0,0,0,NaN
1,20200801,AL,89349,608022,NaN,1488.0,10521.0,NaN,1150.0,NaN,...,697371,23,0,051600f9d45322940b3b932b89df96620784ce06,0,0,0,0,0,NaN
2,20200801,AR,43173,471408,NaN,507.0,2852.0,NaN,NaN,100.0,...,514581,7,0,e19295575aa90b4b79882d8ec3b160aa7f750760,0,0,0,0,0,NaN
3,20200801,AS,0,1267,NaN,NaN,NaN,NaN,NaN,NaN,...,1267,0,0,49561c5d47a0b1611f07f40e32ad91ef8aee2676,0,0,0,0,0,NaN
4,20200801,AZ,177002,771648,NaN,2226.0,11346.0,710.0,NaN,490.0,...,948650,53,86,b75bceda7e20c660fabc4c5f51570bd73b73d690,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200801,AK,3827.0,234806.0,NaN,37.0,NaN,NaN,NaN,3.0,...,238633,1,0,10f9ac619619649c68459b9c290978575b6d3db4,0,0,0,0,0,NaN
1,20200801,AL,89349.0,608022.0,NaN,1488.0,10521.0,NaN,1150.0,NaN,...,697371,23,0,051600f9d45322940b3b932b89df96620784ce06,0,0,0,0,0,NaN
2,20200801,AR,43173.0,471408.0,NaN,507.0,2852.0,NaN,NaN,100.0,...,514581,7,0,e19295575aa90b4b79882d8ec3b160aa7f750760,0,0,0,0,0,NaN
3,20200801,AS,0.0,1267.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1267,0,0,49561c5d47a0b1611f07f40e32ad91ef8aee2676,0,0,0,0,0,NaN
4,20200801,AZ,177002.0,771648.0,NaN,2226.0,11346.0,710.0,NaN,490.0,...,948650,53,86,b75bceda7e20c660fabc4c5f51570bd73b73d690,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-01,AK,3827,234806,NaN,37.0,NaN,NaN,NaN,3.0,NaN,...,238633,1,0,10f9ac619619649c68459b9c290978575b6d3db4,0,0,0,0,0,NaN
2020-08-01,AL,89349,608022,NaN,1488.0,10521.0,NaN,1150.0,NaN,616.0,...,697371,23,0,051600f9d45322940b3b932b89df96620784ce06,0,0,0,0,0,NaN
2020-08-01,AR,43173,471408,NaN,507.0,2852.0,NaN,NaN,100.0,395.0,...,514581,7,0,e19295575aa90b4b79882d8ec3b160aa7f750760,0,0,0,0,0,NaN
2020-08-01,AS,0,1267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1267,0,0,49561c5d47a0b1611f07f40e32ad91ef8aee2676,0,0,0,0,0,NaN
2020-08-01,AZ,177002,771648,NaN,2226.0,11346.0,710.0,NaN,490.0,NaN,...,948650,53,86,b75bceda7e20c660fabc4c5f51570bd73b73d690,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-01,AK,3827.0,234806.0,NaN,37.0,NaN,NaN,NaN,3.0,NaN,...,238633,1,0,10f9ac619619649c68459b9c290978575b6d3db4,0,0,0,0,0,NaN
2020-08-01,AL,89349.0,608022.0,NaN,1488.0,10521.0,NaN,1150.0,NaN,616.0,...,697371,23,0,051600f9d45322940b3b932b89df96620784ce06,0,0,0,0,0,NaN
2020-08-01,AR,43173.0,471408.0,NaN,507.0,2852.0,NaN,NaN,100.0,395.0,...,514581,7,0,e19295575aa90b4b79882d8ec3b160aa7f750760,0,0,0,0,0,NaN
2020-08-01,AS,0.0,1267.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1267,0,0,49561c5d47a0b1611f07f40e32ad91ef8aee2676,0,0,0,0,0,NaN
2020-08-01,AZ,177002.0,771648.0,NaN,2226.0,11346.0,710.0,NaN,490.0,NaN,...,948650,53,86,b75bceda7e20c660fabc4c5f51570bd73b73d690,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE